# <p style="background-color:#56BBD1;font-family:newtimeroman;color:#2E3035;font-size:150%;text-align:center;border-radius:10px 10px;">Animated Plots using Matplotlib Animation</p>

Our world feeds on data. If a picture is worth a thousand words, how valuable can an animation be? By using animated data visualization or videos, our ability to convey an idea can be greatly magnified.

In this project, I will explain 3 different ways on how to generate data animations using Matplotlib and Celluloid packages in Python.

<a id='top'></a>
<div class="list-group" id="list-tab" role="tablist">
<p style="background-color:#56BBD1;font-family:newtimeroman;color:#2E3035;font-size:150%;text-align:center;border-radius:10px 10px;">TABLE OF CONTENTS</p>   

* [1. LOADING PACKAGES AND DATA](#1)
    
* [2. MATPLOTLIB ANIMATION](#2)
    
    * [2.1. ANIMATION 1: ERASING AND REDRAWING](#3)
    
    * [2.2. ANIMATION 2: SETTING THE ARTIST'S DATA](#4)
    
* [3. ANIMATION 3: USING CELLULOID](#5)
          
* [4. CONCLUSION](#6)
  
<a id="1"></a>
# <p style="background-color:#56BBD1;font-family:newtimeroman;color:#2E3035;font-size:150%;text-align:center;border-radius:10px 10px;">Load packages and data</p>

In [ ]:
import numpy as np
import matplotlib.animation as animation
import matplotlib.pyplot as plt
from matplotlib import rc
rc('animation', html='jshtml')
# plt.style.use('ggplot')
# plt.style.use('seaborn')

# Fixing random state for reproducibility
np.random.seed(42)

precomputed_frames = 100
total_frames = 130

# Define "offline" data
x1 = np.linspace(start=0, stop=20, num=precomputed_frames)
y1 = 0.1*(x1 + 3*np.random.random(size=(len(x1))))**2
heights= np.array([[4+2*i for i,j in enumerate(range(precomputed_frames))],
                   [30+0.3*i for i,j in enumerate(range(precomputed_frames))],
                   [4+1*i for i,j in enumerate(range(precomputed_frames))],
                   [40-0.2*i for i,j in enumerate(range(precomputed_frames))]
                   ], dtype=object).T

colors = ['gold', 'yellowgreen', 'lightcoral', 'lightskyblue']
labels = ['L1', 'L2', 'L3', 'L4']
hist_bins = np.linspace(-4, 4, 12)

<a id="2"></a>
# <p style="background-color:#56BBD1;font-family:newtimeroman;color:#2E3035;font-size:150%;text-align:center;border-radius:10px 10px;">Matplotlib Animation</p>

Matplotlib Animation is an animation tool that centers around the matplotlib.animation.Animation base class, which provides a framework around which the animation functionality is built upon. There are to ways to create an animation:

- By using the FuncAnimation interface, an animation is made by repeatedly calling a function func, a function where new data will be plotted. We will use this method when using the Animation module directly.

- By using ArtistAnimation interface, all plotting should already have taken place and the results saved. This is what Celluloid uses under the hood.

Although the basic principle of an animation is an overlay of different shapes and figures, there are different ways in which this can be done. Matplotlib allows the use of blitting, an optimization technique that renders non-changing graphic elements into a background, and only draw the changing elements. Blitting can improve the performance of interactive figures, but can become more complex to use.

By setting blit = False, the simplest way to generate an animated plot is by clearing our figure, and plotting new data. If we don't clear the figure, it can become crowded by the old plots, as can be seen in the example below.

In [ ]:
fig, axes = plt.subplots(nrows=4, ncols=1, figsize=(15,8))
plt.tight_layout()
line, = axes[3].plot([], [], linewidth=5)
for i, ax in enumerate(axes):
    ax.set_ylim(0, 13)
    ax.set_xlim(-1, 10)

axes[0].set_title("Plot 1: Clearing the axes before plotting again using the 'plot' command", fontsize=22, y=0.75)
axes[1].set_title("Plot 2: Same as Plot 1, but setting the x and y-limits for each plot", fontsize=22, y=0.75)
axes[2].set_title("Plot 3: Plotting new data using the 'plot' command without clearing the axes", fontsize=22, y=0.75)
axes[3].set_title("Plot 4: Setting new data directly to the line's artist, WITHOUT using the 'plot' command", fontsize=22, y=0.75)

def animate(i):
    # Plot 1
    axes[0].clear()
    axes[0].set_title("Plot 1: Clearing the axes before plotting again using the 'plot' command", fontsize=22, y=0.75)
    axes[0].plot([i] * 10, np.arange(10), linewidth=5)
    # Plot 2
    axes[1].clear()
    axes[1].set_ylim(0, 13)
    axes[1].set_xlim(-1, 10)
    axes[1].set_title("Plot 2: Same as Plot 1, but setting the x and y-limits for each plot", fontsize=22, y=0.75)
    axes[1].plot([i] * 10, np.arange(10), linewidth=5)
    # Plot 3
    axes[2].plot([i] * 10, np.arange(10), linewidth=5, )
    # Plot 4
    line.set_data([i] * 10, np.arange(10))

plt.close()
    
anim = animation.FuncAnimation(fig, animate, frames=10, interval=200, blit=False)
anim

In [ ]:
# anim.save('matplotlib_animation_example.gif', writer='imagemagick')

Suppose that we want to plot a simple line moving across the screen. In Plot 1, when we clear the plot and the axes before plotting new data, the new plot is always horizontally centered by default, which misses the point of the animation. If we DON'T clear the plot, we get multiples plots as can be seen in Plot 3. We only get the desired plot of a moving line if we clear the previous plot and redefine the axes limits, as can be seen in Plot 2, or by setting the new data directly to the line itself, as was done in Plot 4.

<a id="3"></a>
# <p style="background-color:#56BBD1;font-family:newtimeroman;color:#2E3035;font-size:150%;text-align:center;border-radius:10px 10px;">Animation 1: Erasing and Redrawing</p>

The method of erasing and redrawing a plot is the simplest way to create an animation using only matplotlib. As can be seen from lines 27–37 from the script below, we only need 1 single line of code to plot each subplot individually using pretty much simple and standard plotting functions. The downside is that now our function only accepts a single argument and we need to keep redefining the axes title, limits, and other visual components.

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(10,5))
xlim_list = [(0, 20), (-1, 4), (0, 1), (0, 40), (-4, 4), (0.7, 1.3)]
ylim_list = [(0, 50), (0, 220), (0, 1), (-1.2, 1.2), (0, 130), (0.7, 1.3)]
title_list = [f"Plot {i+1}" for i in range(6)]
plt.tight_layout()

# Allocate space for "live" data
n_sine, n_hist, n_scat = 0.3, 3, 1
data_sine = np.full(shape=(total_frames+1), fill_value=np.nan)
data_hist = np.full(shape=((total_frames+1)*n_hist), fill_value=np.nan)
data_scat = np.full(shape=(2, total_frames*n_scat), fill_value=np.nan)

def animate_simple(k):
    i = k if k < (precomputed_frames-1) else (precomputed_frames-1)
    
    # Simulate data feed by appending new data
    data_sine[k] = n_sine*(k+1)
    data_hist[n_hist*k:n_hist*(k+1)] = np.random.randn(n_hist)    
    data_scat[:, n_scat*k:n_scat*(k+1)] = np.random.default_rng().lognormal(mean=0, sigma=0.1, size=(2,n_scat))
    
    # Dynamically adjust the x-axis limits for Plot 4
    xlim_list[3] = (0, np.nanmax(data_sine))

    # Clear axis, define title and axis limits
    for (ax, title, xlim, ylim) in zip(axes.flatten(), title_list, xlim_list, ylim_list):
        ax.clear()
        ax.set_title(title)
        ax.set_xlim(xlim)
        ax.set_ylim(ylim)

    # Plot 1
    axes[0][0].plot(x1[:i], y1[:i])
    # Plot 2
    axes[0][1].bar(labels, heights[i], color=colors)
    # Plot 3
    axes[0][2].pie(heights[i], colors=colors, labels=labels, autopct='%1.1f%%')
    # Plot 4
    axes[1][0].plot(data_sine, np.sin(data_sine))
    # Plot 5
    axes[1][1].hist(data_hist, bins=hist_bins, ec='black')
    # Plot 6
    axes[1][2].scatter(x=data_scat[0, :], y=data_scat[1, :], c="green", alpha=0.5)

# Code to prevent a static image from being generated from this cell
plt.close()

In [ ]:
anim1 = animation.FuncAnimation(fig, animate_simple, frames=total_frames, interval=40, blit=False)
anim1

In [ ]:
# anim1.save('matplotlib_simple_animation.gif', writer='imagemagick')

<a id="4"></a>
# <p style="background-color:#56BBD1;font-family:newtimeroman;color:#2E3035;font-size:150%;text-align:center;border-radius:10px 10px;">Animation 2: Setting the artist's data</p>

Can we avoid the need to clear the figure before plotting again? Well, yes… but there's a catch. We can set blit=True and instead of drawing the whole plot again, we modify what is already in the figure. For a simple line or a bar plot this can be easy, but for other plots (pie plot or Plot 3, for instance) this can become quite cumbersome, since we have to calculate the angles and other features of the plot.

In [ ]:
fig = plt.figure(figsize=(10,5))
ax1 = fig.add_subplot(231, title="Plot 1", xlim=(0, 20), ylim=(0, 50))
ax2 = fig.add_subplot(232, title="Plot 2", ylim=(0, np.max(heights)+20))
ax3 = fig.add_subplot(233, title="Plot 3")
ax4 = fig.add_subplot(234, title="Plot 4", ylim=(-1.2, 1.2))
ax5 = fig.add_subplot(235, title="Plot 5", xlim=(-4, 4), ylim=(0, 130))
ax6 = fig.add_subplot(236, title="Plot 6", xlim=(0.7, 1.3), ylim=(0.7, 1.3))
plt.tight_layout()

line1, = ax1.plot([], [])
barcollection = ax2.bar(labels, heights[0], color=colors)
wedges, pie_lbs, pie_pct = ax3.pie(heights[0], colors=colors, labels=labels, autopct='%1.1f%%')
line2, = ax4.plot([], [])
_, _, hist_container = ax5.hist([], bins=hist_bins, ec="black")
hist_container = list(hist_container)
scatter, = ax6.plot([], [], marker='o', color='green', alpha=0.5, linestyle='')

# Allocate space for "live" data
n_sine, n_hist, n_scat = 0.3, 3, 1
data_sine = np.full(shape=(total_frames+1), fill_value=np.nan)
data_hist = np.full(shape=((total_frames+1)*n_hist), fill_value=np.nan)
data_scat = np.full(shape=(2, total_frames*n_scat), fill_value=np.nan)

def animate_artist(k):
    i = k if k < (precomputed_frames-1) else (precomputed_frames-1)
    
    # Plot 1
    line1.set_data(x1[:i], y1[:i])

    # Plot 2
    for j, b in enumerate(barcollection):
        b.set_height(heights[i][j])

    # Plot 3
    cumsum_heights = np.insert(np.cumsum(heights[i]), 0, 0)
    h_min, h_max = np.min(cumsum_heights), np.max(cumsum_heights)
    theta = (cumsum_heights - h_min)*360/(h_max-h_min)
    for j, (w, p, l) in enumerate(zip(wedges, pie_pct, pie_lbs)):
        w.set_theta1(theta[j])
        w.set_theta2(theta[j+1])
        radians = np.radians((theta[j+1] - theta[j])/2 + theta[j])
        p.set_x(0.7*np.cos((radians)))
        p.set_y(0.7*np.sin((radians)))
        p.set_text(f"{heights[i][j]/h_max*100 :.1f}%")
        l.set_x(1.2*np.cos((radians)))
        l.set_y(1.2*np.sin((radians)))
    
    # Simulate data feed by appending new data
    data_sine[k] = n_sine*k
    data_hist[n_hist*k:n_hist*(k+1)] = np.random.randn(n_hist)    
    data_scat[:, n_scat*k:n_scat*(k+1)] = np.random.default_rng().lognormal(mean=0, sigma=0.1, size=(2,n_scat))
    
    # Plot 4
    ax4.set_xlim(0, np.nanmax(data_sine)+1)
    line2.set_data(data_sine, np.sin(data_sine))
    
    # Plot 5
    n, _ = np.histogram(data_hist[~np.isnan(data_hist)], bins=hist_bins)
    # for count, rect in zip(n, hist_container.patches):
    for count, rect in zip(n, hist_container):
        rect.set_height(count)

    # Plot 6
    scatter.set_data([data_scat[0, :], data_scat[1, :]])

    return line1, 
    # return (line1, barcollection.patches[0], barcollection.patches[1], barcollection.patches[2], barcollection.patches[3], wedges[0], wedges[1], wedges[2], wedges[3],  line2, hist_container[0], hist_container[1], hist_container[2], hist_container[3], hist_container[4], hist_container[5], hist_container[6], hist_container[7], hist_container[8], hist_container[9], hist_container[10], scatter)
    
# Code to prevent a static image from being generated from this cell
plt.close()

In [ ]:
anim2 = animation.FuncAnimation(fig, animate_artist, frames=total_frames, interval=40, blit=True)
anim2

In [ ]:
# anim2.save('matplotlib_artist_animation.gif', writer='imagemagick')

<a id="5"></a>
# <p style="background-color:#56BBD1;font-family:newtimeroman;color:#2E3035;font-size:150%;text-align:center;border-radius:10px 10px;">Animation 3: Using Celluloid</p>
However, maybe you ALREADY have written the code to plot your data and you don't want to adapt it. Can we still generate an animation from it? Yes! Using the celluloid module, we can plot our data visualization just as we usually do and take a "picture" from it every time we do so inside a loop.

First, we have to import the celluloid module:

In [ ]:
!pip install celluloid

Now, let's check how our animation function looks like:

In [ ]:
from celluloid import Camera

fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(10,5))
xlim_list = [(0, 20), (-1, 4), (0, 1), (0, 40), (-4, 4), (0.7, 1.3)]
ylim_list = [(0, 50), (0, 220), (0, 1), (-1.2, 1.2), (0, 130), (0.7, 1.3)]
title_list = [f"Plot {i+1}" for i in range(6)]
plt.tight_layout()

# Define title and axis limits
for (ax, title, xlim, ylim) in zip(axes.flatten(), title_list, xlim_list, ylim_list):
    ax.set_title(title)
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)

# Allocate space for "live" data
n_sine, n_hist, n_scat = 0.3, 3, 1
data_sine = np.full(shape=(total_frames+1), fill_value=np.nan)
data_hist = np.full(shape=((total_frames+1)*n_hist), fill_value=np.nan)
data_scat = np.full(shape=(2, total_frames*n_scat), fill_value=np.nan)

def animate_celluloid(i):
    i = k if k < (precomputed_frames-1) else (precomputed_frames-1)

    # Simulate data feed by appending new data
    data_sine[k] = n_sine*(k+1)
    data_hist[n_hist*k:n_hist*(k+1)] = np.random.randn(n_hist)
    data_scat[:, n_scat*k:n_scat*(k+1)] = np.random.default_rng().lognormal(mean=0, sigma=0.1, size=(2,n_scat))

    # Dynamically adjust the x-axis limits for Plot 4
    # Spoiler: This doesn't work for for celluloid
    xlim_list[3] = (0, np.nanmax(data_sine))
    axes[1][0].set_xlim(xlim_list[3])

    # Plot 1
    axes[0][0].plot(x1[:i], y1[:i], color='blue')
    # Plot 2
    axes[0][1].bar(labels, heights[i], color=colors)
    # Plot 3
    axes[0][2].pie(heights[i], colors=colors, labels=labels, autopct='%1.1f%%')
    # Plot 4
    axes[1][0].plot(data_sine, np.sin(data_sine))
    # Plot 5
    axes[1][1].hist(data_hist, bins=hist_bins, ec='black', fc="blue")
    # Plot 6
    axes[1][2].scatter(x=data_scat[0, :], y=data_scat[1, :], c="green", alpha=0.5)


camera = Camera(fig)
for k in range(total_frames):
    animate_celluloid(k)
    camera.snap()

# Code to prevent a static image from being generated from this cell
plt.close()

In [ ]:
anim3 = camera.animate(interval=50, blit=False)
anim3

As can be seen, this function is a straightforward plotting function without the need of fancy code or erasing any figure. Just like in Animation 1, we can plot each subplot using a single line of code, and we don't need to erase anything!

Still… there are some caveats.

I have intentionally avoided to define the line color in Plot 4 to illustrate that it is being constantly being redraw in different colors for each frame, so to maintain consistency we need to define the plot color, as was done with the other subplots.

Besides, Plot 4 also begins with the x-axis already expanded, despite the code adjusting the limit for each frame. And this is in fact one of the limitations of celluloid: you can't have dynamic axes by using this library.

Despite the fixed axis limitation, this method of animating data is simpler than Animation 1 because we don't need to reconfigure the axis, titles, legends and etc for each plot every time it is plotted. Besides, since we're calling the plotting function animate_celluloid directly, we can easily provide whatever arguments that this function needs.

In [ ]:
# anim3.save('celluloid_animation.gif', dpi=100)

<a id="6"></a>
# <p style="background-color:#56BBD1;font-family:newtimeroman;color:#2E3035;font-size:150%;text-align:center;border-radius:10px 10px;">Conclusions</p>
Animation can be powerful. It is after all another dimension in which data can be plotted, and can be specifically useful to plot the evolution or progress of a set of data.

There are two ways to make animations using matplotlib. By directly modifying the data in the plot, powerful and computationally efficient animations can be made, although the complexity of the underlying code can become increasingly higher. By erasing and redrawing the plots, only slight modifications to the plotting functions are necessary, but they are necessary nonetheless. By using celluloid we can use our plotting code as it is to generate animations, although there are some limitations that can impact the results.